In [ ]:
import ase
import ase.io as ase_io
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from copy import deepcopy

from dscribe.descriptors import SOAP

In [2]:
def rasterize(cf):
    for c in cf.collections:
        c.set_rasterized(True)
    return cf

In [ ]:
fw = ase_io.read("chemrev_nuprime-theta-grid_computed.xyz", index=":")
nrg = np.loadtxt("pswater-ipi.out")[:, 0]
for f, n in zip(fw, nrg[1:]):
    f.info["energy"] = n

y, yl, rt, lw = [], [], [], []
for w in fw:
    rt.append([w.info["OH1"], w.info["OH2"], w.info["HOH"]])
    y.append(w.info["energy"] * 27.211386)     
    lw.append(w)

rt = np.asarray(rt)
y = np.asarray(y)
#yl = np.asarray(yl)
ntot = len(y)
szgrid = (11, 11)

In [ ]:
HYPERS_DSCRIBE = {
    "species": ["H", "O"],
    "r_cut": 2.0,      
    "n_max": 8,        # = max_radial
    "l_max": 6,        # = max_angular
    "sigma": 0.5,      # = gaussian_sigma_constant
    "rbf": "gto",      # = radial_basis: 'GTO'
    "periodic": False, 
    "average": "off",  
    "compression": {"mode": "off"},  
}

soap = SOAP(**HYPERS_DSCRIBE)

In [ ]:
centers_list = [[i for i, a in enumerate(w) if a.symbol == "O"] for w in fw]

In [ ]:
X = soap.create(fw, centers=centers_list)

print("Total number of structures:", len(fw))
print("Total number of centers (should equal the number of structures, since H2O has only one O atom):", 
      sum(len(c) for c in centers_list))
print("Shape of SOAP feature matrix:", X.shape)

In [ ]:
import numpy as np
import pandas as pd

X_2d = X.reshape(X.shape[0], -1)     

print("X_2d shape:", X_2d.shape)    
print("y shape:", y.shape)          


X_2d shape: (121, 952)
y shape: (121,)


In [14]:
pd.DataFrame(X_2d).to_csv("X_soap.csv", index=False)
pd.DataFrame({"energy_eV": y}).to_csv("y_energy.csv", index=False)